# TI3130: Data Wrangling Lab &mdash; Solutions (variant B)
**Julián Urbano &mdash; November 2021**

In [1]:
import sys
import numpy as np
import pandas as pd
print("python ", sys.version,
      "\nnumpy", np.__version__,
      "\npandas", pd.__version__)

python  3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)] 
numpy 1.21.4 
pandas 1.3.4


In [2]:
listings = pd.read_csv("airbnb_listings.csv")
reviews = pd.read_csv("airbnb_reviews.csv")

a) What hosts have listings in the Alexanderplatz `neighborhood` that do not require more than 2 nights?

In [8]:
listings \
    .query('neighborhood == "Alexanderplatz" & minimum_nights <= 2') \
    .loc[: , 'host_id'] \
    .drop_duplicates()

48        261953
90        632174
94        681637
98        722460
120       833209
          ...   
4590    36961901
4686    37597410
4701    29513589
4776    17698129
4987    39213221
Name: host_id, Length: 76, dtype: int64

b) How many users have submitted at least two reviews?

In [11]:
reviews \
    .groupby(['reviewer_id']) \
    .aggregate(n = ('reviewer_id', 'count')) \
    .query('n >= 2')

,n
reviewer_id,
2915,2
11241,2
12602,2
43010,2
64564,2
...,...
159590484,2
169079459,2
185861725,2


c) What reviewers have reviewed the same listing more than three times?

In [12]:
reviews \
    .groupby(['listing_id', 'reviewer_id']) \
    .aggregate(n = ('reviewer_id', 'count')) \
    .query('n > 3')

n
listing_id reviewer_id   
107968     2648035      4
171746     95627040     4
187206     5023231      4
           38158204     4
220808     30777232     4
230824     23863556     4
237038     1442940      4
238396     11379932     4
385059     38101555     9
613219     115045439    4
701909     13127535     7
719195     11354354     4
742211     23180684     4

d) Select the top 10 hosts with most reviews. Show their `host_id`, `host_name` and number of `reviews`.

In [26]:
pd.merge(listings, reviews, how = 'inner', left_on = 'id', right_on = 'listing_id') \
    .groupby(['host_id', 'host_name'], as_index=False) \
    .aggregate(reviews = ('id_y', 'count')) \
    .sort_values('reviews', ascending=False) \
    .head(10)

,host_id,host_name,reviews
63,286494,Project A,957
446,2547002,Studio A,900
33,163384,Alan & Kasia,848
238,1250154,Henrik,799
157,868282,Frank,783
21,98237,Damian,623
178,929943,Reineldis & Fabian,613
187,955509,Karsten,586
343,1654885,Claudius,573
112,632174,Stéphanie,443


e) Sort `neighborhood`s in descending order by the range of their prices (ie. maximum minus minimum). Consider only listings whose `price` is at least 20€/night, and show only neighborhoods with a range different from 0.

In [39]:
listings \
    .query('price >= 20') \
    .groupby('neighborhood') \
    .apply(lambda x: pd.Series({'range': x.price.max() - x.price.min()})) \
    .query('range != 0') \
    .sort_values('range', ascending=False)

,range
neighborhood,
Helmholtzplatz,4220
Tempelhofer Vorstadt,2480
Pankow Zentrum,776
Wedding Zentrum,630
Neue Kantstraße,580
...,...
Marzahn-Süd,9
Buckow Nord,7
Niederschöneweide,6


f) Sort `neighborhoods` by their listing density, in descending order. Density may be calculated as the number of listings by surface. Surface may be calculated as the range of `latitude` multiplied by the range of `longitude`. Tip: rows with an `Inf` value may be removed by first [replacing](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) `np.inf` values with `np.nan` and then calling `dropna`.

In [96]:
nborhoods = listings \
    .groupby('neighborhood', as_index=False) \
    .apply(lambda x: pd.Series({'listings': x.id.count(), 'range_lat': x.latitude.max() - x.latitude.min(), 'range_long': x.longitude.max() - x.longitude.min()})) \

nborhoods['surface'] = nborhoods['range_lat'] * nborhoods['range_long']

nborhoods['density'] = nborhoods['listings'] / nborhoods['surface']

nborhoods \
    .replace(np.inf, np.nan) \
    .dropna() \
    .loc[:, ['neighborhood', 'density']]
    

,neighborhood,density
1,Albrechtstr.,3.810019e+04
2,Alexanderplatz,1.295363e+05
3,Alt Treptow,2.715418e+05
4,Alt-Hohenschönhausen Nord,1.852787e+06
5,Alt-Hohenschönhausen Süd,2.131289e+04
...,...,...
123,Wilhelmstadt,7.202331e+04
124,Zehlendorf Nord,4.899799e+03
125,Zehlendorf Südwest,4.587605e+03
126,nördliche Luisenstadt,5.467421e+05


g) Create a data frame where rows are `host_ids`, columns are `neighborhood_groups`, and values are the mean `price` of the listings of that host in that neighborhood group. Round the price to full euros.

In [118]:
nhood_per_host = listings \
    .groupby(['host_id', 'neighborhood_group'], as_index = False) \
    .aggregate(mean_price = ('price', 'mean'))
nhood_per_host['mean_price'] = nhood_per_host['mean_price'].round()
nhood_per_host = nhood_per_host.pivot(index = 'host_id',
          columns = 'neighborhood_group', values = 'mean_price')

nhood_per_host

neighborhood_group,Charlottenburg-Wilm.,Friedrichshain-Kreuzberg,Lichtenberg,Marzahn - Hellersdorf,Mitte,Neukölln,Pankow,Reinickendorf,Spandau,Steglitz - Zehlendorf,Tempelhof - Schöneberg,Treptow - Köpenick
host_id,,,,,,,,,,,,
2217,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2986,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN
3718,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN
4108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN
11015,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
39219660,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39222511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN
39230948,NaN,NaN,NaN,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


h) Same as g), but show only cases where the average price is cheaper than 30€/night.

In [121]:
nhood_per_host = listings \
    .groupby(['host_id', 'neighborhood_group'], as_index = False) \
    .aggregate(mean_price = ('price', 'mean'))
nhood_per_host['mean_price'] = nhood_per_host['mean_price'].round()
nhood_per_host = nhood_per_host.query('mean_price < 30')
nhood_per_host = nhood_per_host.pivot(index = 'host_id',
          columns = 'neighborhood_group', values = 'mean_price')

nhood_per_host

neighborhood_group,Charlottenburg-Wilm.,Friedrichshain-Kreuzberg,Lichtenberg,Marzahn - Hellersdorf,Mitte,Neukölln,Pankow,Reinickendorf,Spandau,Steglitz - Zehlendorf,Tempelhof - Schöneberg,Treptow - Köpenick
host_id,,,,,,,,,,,,
2986,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN
4108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN
12723,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN
130019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN
159734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
39137222,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39152545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN
39156665,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN


i) Take the result of g) and turn it into long format. Drop any rows with missing values.

In [124]:
nhood_per_host = listings \
    .groupby(['host_id', 'neighborhood_group'], as_index = False) \
    .aggregate(mean_price = ('price', 'mean'))
nhood_per_host['mean_price'] = nhood_per_host['mean_price'].round()
nhood_per_host = nhood_per_host.query('mean_price < 30')
nhood_per_host = nhood_per_host.pivot(index = 'host_id',
          columns = 'neighborhood_group', values = 'mean_price')
nhood_per_host = nhood_per_host.reset_index()
nhood_per_host = nhood_per_host.melt(id_vars = 'host_id', var_name = 'neighborhood_group', value_name = 'mean_price')
nhood_per_host = nhood_per_host.dropna()

nhood_per_host

,host_id,neighborhood_group,mean_price
12,813767,Charlottenburg-Wilm.,23.0
18,901878,Charlottenburg-Wilm.,28.0
26,1135836,Charlottenburg-Wilm.,29.0
42,1787661,Charlottenburg-Wilm.,29.0
67,2915907,Charlottenburg-Wilm.,25.0
...,...,...,...
6943,12856547,Treptow - Köpenick,20.0
7031,19347591,Treptow - Köpenick,29.0
7103,24064848,Treptow - Köpenick,23.0
7135,26645527,Treptow - Köpenick,27.0
